In [22]:
from keras.layers.core import Activation, Dense, Dropout
from keras.layers import concatenate,Reshape,Add,LSTM,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential,Model
from keras import Model
from keras import Input

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from keras.initializers import Constant

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split


## Define parameters

In [108]:
seqLen=100
dateShape=(10,)
signShape=(10,)
textShape=(10,10)

## Define structure

### Date input

In [109]:
inputDate=Input(shape=dateShape)
denseD1=Dense(8,activation="relu")(inputDate)
outputD=Dense(16,activation="relu")(denseD1)

### Sign input

In [110]:
inputSign=Input(shape=signShape)
denseS1=Dense(8,activation="relu")(inputSign)
outputS=Dense(16,activation="relu")(denseS1)

### Text input

In [111]:
inputText=Input(shape=textShape)

### Concatenate date and sign

In [112]:
combined=concatenate([outputF,outputS])

## input->add tanh->lstm->mul sigm->lstm

### Define tanh dense

In [113]:
denseTanh=Dense(100,activation='tanh')(combined)
reshapedTanh=Reshape(textShape)(denseTanh)

### Define sigmoid dense

In [114]:
denseSigmoid=Dense(100,activation='sigmoid')(combined)
reshapedSigmoid=Reshape(textShape)(denseSigmoid)

### Add 

In [115]:
add1=Add()([reshapedTanh,inputText])
lstm1=LSTM(100)(add1)
reshapedLstm=Reshape(textShape)(lstm1)
mul1=Multiply()([reshapedLstm,reshapedSigmoid])
lstm2=LSTM(100)(mul1)

## Define model

In [1]:
#model=Model(inputs=[inputDate,inputSign,inputText],outputs=[lstm2])

размер словаря - 84267 слов, максимальная длинна текста - 398 символов


In [6]:
data=np.array(pd.read_csv('data.csv'))

In [7]:
def processText(data):
    tokens=[]
    for line in data:
        newToken=text_to_word_sequence(text=line[2],filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff',
                                  lower=True,split=' ')
        tokens.append(newToken)
    return tokens

In [8]:
tokens=processText(data)

In [9]:
w2v = Word2Vec.load("word2vec.model")

In [32]:
num_words = 84267
tokenizer = Tokenizer(
    num_words=num_words,
    filters='!"#$%&amp;()*+,-—./:;&lt;=>?@[\\]^_`{|}~\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    char_level=False) #, oov_token='unknown'
tokenizer.fit_on_texts(tokens) #формируем токены на основе частотности в нашем текст
sequences = np.array(tokenizer.texts_to_sequences(tokens))

In [33]:
sequences[0]

[293,
 448,
 66,
 8385,
 4715,
 15933,
 22,
 8,
 10529,
 336,
 34,
 10057,
 25,
 127,
 3,
 16752,
 7,
 12640,
 169,
 755,
 254,
 43,
 3549,
 279,
 4619,
 953,
 12,
 2,
 614]

In [ ]:
def tokenizeIt(review_lines):
  num_words = 20000 #число слов, которые войдут в словарь частотности
  print("Размер выборки:", len(review_lines))
  tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&amp;()*+,-—./:;&lt;=>?@[\\]^_`{|}~\t\n\xa0\ufeff😏🤔😟👍👎🏻🤘🥺😦😲🤪👋☝️🖖✌️👌👏💪🏻🤝‍♂️♂️🏠🚗😈🤦🤦‍♂️‼️♥❤️💗😻😻😍🤩😆😎😜🔥😨😱😭😂🤣😀😃😂😁😊😉🤗💯', lower=True, split=' ', char_level=False) #, oov_token='unknown'
  tokenizer.fit_on_texts(review_lines) #формируем токены на основе частотности в нашем текст
  sequences = tokenizer.texts_to_sequences(review_lines)
  max_len = len(max(sequences, key = len)) #максимальная длина предложения (длина развернутой RNN сети)
  word_index = tokenizer.word_index
  num_words = len(tokenizer.word_index) + 1
  print("Максимальная длина отзыва:", max_len, "слова.")
  print("Уникальных токенов:", len(word_index))
  print("Номер тестового слова:", word_index['водитель']) #Проверяем индекс выбранного слова
  print("Кол-во слов:", num_words)
  return max_len, num_words, sequences, word_index, tokenizer

In [ ]:
max_len, num_words, sequences, word_index, tokenizer = tokenizeIt(writers_lines)
print(word_index)
print(sequences)

In [ ]:
x_train_pad = pad_sequences(sequences = sequences, maxlen = max_len) #, padding='post', truncating='post') #все предложения длиной &lt;max_len дополняем нулями в конце, >max_len - отбрасываем
print(x_train_pad[5])
print(x_train_pad[5].shape)
print("x_train shape:", x_train_pad.shape)

In [ ]:
#Подготавливаем embedding слой для использования в модели
#В матрице embedding номер слова заменяется на вектор из модели word2vec  
def getEmbeddingLayer(num_words, embedding_size, max_length, tokenizer, word2vec, Trainable = False):
  print("Размер embedding матрицы:", num_words, "x", embedding_size)
  embedding_matrix = np.zeros((num_words, embedding_size))
  for word, i in tokenizer.word_index.items():
    if i > num_words: #если индекс превышает кол-во слов в словаре, то скипаем  
      continue
    embedding_vector = w2v[word] #получаем вектор соответствущий слову в модели word2vec
    if embedding_vector is not None:  #если слово отсутствует в словаре word2vec, то оно в матрице np.zeroes останется равным 0
      embedding_matrix[i] = embedding_vector #если слово найдено в словаре токенизатора, то в embedding_matrix проставляем вектор соответствующий слову
      embedding_layer = Embedding(input_dim = num_words, 
                              output_dim = embedding_size, 
                              embeddings_initializer = Constant(embedding_matrix),
                              input_length = max_length, 
                              trainable = Trainable)    
  return embedding_layer, embedding_matrix  

In [ ]:
writers_embedding_layer, writers_embedding_matrix = getEmbeddingLayer(
    num_words,
    EMBEDDING_DIM,
    max_len, 
    tokenizer,
    writers_w2v, 
    Trainable = False)

In [ ]:
num_classes = 5
#Подготовка обучающей и проверочной выборки
x_train_, x_test, y_train, y_test = train_test_split(x_train_pad, y_train_all, test_size = 0.2) #, random_state = 42)
z_train = y_train[:, num_classes:8]
z_test = y_test[:, num_classes:8]
y_train = y_train[:, :num_classes]
y_test = y_test[:, :num_classes]

In [ ]:
def buildModel(embedding_layer):
  modelGRU = Sequential()
  #embedding_layer = w2v.wv.get_keras_embedding(train_embeddings=False)
  modelGRU.add(embedding_layer)
  #modelGRU.add(Embedding(num_words, embedding_size))
  modelGRU.add(SpatialDropout1D(0.2))
  modelGRU.add(Bidirectional(GRU(40, return_sequences=True)))
  modelGRU.add(Bidirectional(GRU(40)))
  #modelGRU.add(LSTM(8,return_sequences=True ))
  #modelGRU.add(LSTM(8))
  modelGRU.add(Dropout(0.2))
  modelGRU.add(Dense(64,activation = 'relu'))
  modelGRU.add(Dropout(0.2))
  modelGRU.add(Dense(num_classes,activation = 'sigmoid'))
  modelGRU.summary()
  return modelGRU

In [ ]:
modelGRU = buildModel(embedding_layer)
#modelGRU.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=1e-4))
modelGRU.compile(loss='binary_crossentropy', metrics=[AUC(name='auc')], optimizer=Adam(lr=1e-4))
historyGRU = modelGRU.fit(X_train, Y_train, batch_size=64, epochs=20, validation_data=(X_test, Y_test))